# GamePulse Data Analysis - Getting Started

This notebook provides an introduction to analyzing GamePulse data, including:
- Database connection setup
- Data pipeline overview
- Social post lifecycle tracking
- Match analysis

## 1. Database Connection Setup

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sqlmodel import create_engine, Session, text

# Set plotting style
sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = [12, 6]

# Build connection string from environment variables
DATABASE_URL = (
    f"postgresql+psycopg://{os.environ.get('POSTGRES_USER', 'postgres')}:"
    f"{os.environ.get('POSTGRES_PASSWORD', 'changethis')}@"
    f"{os.environ.get('POSTGRES_SERVER', 'db')}:"
    f"{os.environ.get('POSTGRES_PORT', '5432')}/"
    f"{os.environ.get('POSTGRES_DB', 'app')}"
)

engine = create_engine(DATABASE_URL)
print(f"Connected to database: {os.environ.get('POSTGRES_DB', 'app')}")

In [ ]:
# Helper function for running queries
def query(sql: str) -> pd.DataFrame:
    """Execute SQL query and return results as DataFrame."""
    with Session(engine) as session:
        return pd.read_sql(text(sql), session.connection())

## 2. Data Pipeline Overview

GamePulse ingests social media data through this pipeline:
```
Raw Sources → Staging → Fact Tables
─────────────────────────────────────
raw_reddit_posts    ↘
                     → stg_social_posts → fact_social_sentiment
raw_bluesky_posts   ↗
```

In [ ]:
# Get row counts at each stage of the pipeline
pipeline_counts = query("""
    SELECT 'raw_reddit_posts' as stage, 1 as stage_order, COUNT(*) as row_count
    FROM raw_reddit_posts
    UNION ALL
    SELECT 'raw_bluesky_posts', 2, COUNT(*) FROM raw_bluesky_posts
    UNION ALL
    SELECT 'stg_social_posts', 3, COUNT(*) FROM stg_social_posts
    UNION ALL
    SELECT 'fact_social_sentiment', 4, COUNT(*) FROM fact_social_sentiment
    ORDER BY stage_order
""")

print("=== Pipeline Row Counts ===")
pipeline_counts[["stage", "row_count"]]

In [ ]:
# Visualize pipeline funnel
fig = go.Figure(
    go.Funnel(
        y=pipeline_counts["stage"],
        x=pipeline_counts["row_count"],
        textinfo="value+percent initial",
    )
)
fig.update_layout(title="Social Data Pipeline Funnel", height=400)
fig.show()

In [ ]:
# Data freshness - when was each table last updated?
freshness = query("""
    SELECT 'raw_reddit_posts' as table_name,
           MAX(fetched_at) as last_updated,
           COUNT(*) as total_rows,
           COUNT(*) FILTER (WHERE fetched_at > NOW() - INTERVAL '1 hour') as last_hour
    FROM raw_reddit_posts
    UNION ALL
    SELECT 'raw_bluesky_posts', MAX(fetched_at), COUNT(*),
           COUNT(*) FILTER (WHERE fetched_at > NOW() - INTERVAL '1 hour')
    FROM raw_bluesky_posts
    UNION ALL
    SELECT 'stg_social_posts', MAX(processed_at), COUNT(*),
           COUNT(*) FILTER (WHERE processed_at > NOW() - INTERVAL '1 hour')
    FROM stg_social_posts
""")

print("=== Data Freshness ===")
freshness

## 3. Social Post Lifecycle

Track posts through their journey:
1. **Ingested**: Raw posts pulled from Reddit/Bluesky
2. **Processed**: Posts transformed and staged
3. **Matched**: Posts successfully linked to games
4. **Discarded**: Posts that couldn't be matched

In [ ]:
# Lifecycle breakdown by platform
lifecycle = query("""
    WITH raw_counts AS (
        SELECT 'reddit' as platform, COUNT(*) as ingested FROM raw_reddit_posts
        UNION ALL
        SELECT 'bluesky', COUNT(*) FROM raw_bluesky_posts
    ),
    staged_counts AS (
        SELECT platform, COUNT(*) as processed
        FROM stg_social_posts
        GROUP BY platform
    ),
    matched_counts AS (
        SELECT s.platform, COUNT(*) as matched
        FROM fact_social_sentiment f
        JOIN stg_social_posts s ON f.social_post_key = s.social_post_key
        GROUP BY s.platform
    )
    SELECT
        r.platform,
        r.ingested,
        COALESCE(s.processed, 0) as processed,
        COALESCE(m.matched, 0) as matched,
        COALESCE(s.processed, 0) - COALESCE(m.matched, 0) as unmatched
    FROM raw_counts r
    LEFT JOIN staged_counts s ON r.platform = s.platform
    LEFT JOIN matched_counts m ON r.platform = m.platform
""")

print("=== Social Post Lifecycle by Platform ===")
lifecycle

In [ ]:
# Calculate conversion rates
if not lifecycle.empty and lifecycle["ingested"].sum() > 0:
    total_ingested = lifecycle["ingested"].sum()
    total_processed = lifecycle["processed"].sum()
    total_matched = lifecycle["matched"].sum()

    print("=== Conversion Rates ===")
    print(f"Ingested → Processed: {total_processed / total_ingested * 100:.1f}%")
    if total_processed > 0:
        print(f"Processed → Matched:  {total_matched / total_processed * 100:.1f}%")
    print(f"Overall (Ingested → Matched): {total_matched / total_ingested * 100:.1f}%")
else:
    print("No data available yet. Run the Dagster pipelines to ingest data.")

In [ ]:
# Visualize lifecycle as stacked bar
if not lifecycle.empty and lifecycle["ingested"].sum() > 0:
    lifecycle_melted = lifecycle.melt(
        id_vars=["platform"],
        value_vars=["matched", "unmatched"],
        var_name="status",
        value_name="count",
    )

    fig = px.bar(
        lifecycle_melted,
        x="platform",
        y="count",
        color="status",
        title="Post Lifecycle: Matched vs Unmatched by Platform",
        color_discrete_map={"matched": "#2ecc71", "unmatched": "#e74c3c"},
    )
    fig.show()

## 4. Volume Trends Over Time

In [ ]:
# Daily ingestion volume (last 30 days)
daily_volume = query("""
    SELECT
        DATE(fetched_at) as date,
        'reddit' as source,
        COUNT(*) as posts
    FROM raw_reddit_posts
    WHERE fetched_at > NOW() - INTERVAL '30 days'
    GROUP BY DATE(fetched_at)
    UNION ALL
    SELECT
        DATE(fetched_at),
        'bluesky',
        COUNT(*)
    FROM raw_bluesky_posts
    WHERE fetched_at > NOW() - INTERVAL '30 days'
    GROUP BY DATE(fetched_at)
    ORDER BY date, source
""")

if not daily_volume.empty:
    fig = px.line(
        daily_volume,
        x="date",
        y="posts",
        color="source",
        title="Daily Ingestion Volume (Last 30 Days)",
        markers=True,
    )
    fig.show()
else:
    print("No data in the last 30 days.")

## 5. Match Analysis

Analyze how well social posts are matched to games.

In [ ]:
# Match rates by subreddit (from raw posts)
subreddit_match_rates = query("""
    SELECT
        subreddit,
        COUNT(*) as total_posts,
        COUNT(*) FILTER (WHERE matched_to_game = true) as matched_posts,
        ROUND(COUNT(*) FILTER (WHERE matched_to_game = true)::numeric / COUNT(*) * 100, 1) as match_rate_pct
    FROM raw_reddit_posts
    GROUP BY subreddit
    HAVING COUNT(*) > 10
    ORDER BY match_rate_pct DESC
""")

if not subreddit_match_rates.empty:
    print("=== Match Rates by Subreddit ===")
    display(subreddit_match_rates)
else:
    print("No subreddit data available.")

In [ ]:
# Game coverage - how many games have social data?
game_coverage = query("""
    WITH game_post_counts AS (
        SELECT
            g.game_key,
            g.game_date,
            COUNT(f.social_post_key) as post_count
        FROM fact_game g
        LEFT JOIN fact_social_sentiment f ON g.game_key = f.game_key
        WHERE g.game_date >= CURRENT_DATE - INTERVAL '30 days'
        GROUP BY g.game_key, g.game_date
    )
    SELECT
        COUNT(*) FILTER (WHERE post_count > 0) as games_with_posts,
        COUNT(*) FILTER (WHERE post_count = 0) as games_without_posts,
        COUNT(*) as total_games,
        ROUND(COUNT(*) FILTER (WHERE post_count > 0)::numeric / NULLIF(COUNT(*), 0) * 100, 1) as coverage_pct
    FROM game_post_counts
""")

print("=== Game Coverage (Last 30 Days) ===")
game_coverage

## 6. Dimensional Data Overview

In [ ]:
# Teams overview
teams = query("""
    SELECT
        sport,
        COUNT(*) as team_count,
        COUNT(*) FILTER (WHERE primary_color IS NOT NULL) as with_colors,
        COUNT(*) FILTER (WHERE array_length(aliases, 1) > 0) as with_aliases
    FROM dim_team
    GROUP BY sport
    ORDER BY team_count DESC
""")

print("=== Teams by Sport ===")
teams

In [ ]:
# Games by sport and status
games = query("""
    SELECT
        sport,
        COUNT(*) as total_games,
        COUNT(*) FILTER (WHERE game_date >= CURRENT_DATE) as upcoming,
        COUNT(*) FILTER (WHERE game_date < CURRENT_DATE) as completed
    FROM fact_game
    GROUP BY sport
    ORDER BY total_games DESC
""")

print("=== Games by Sport ===")
games

## 7. Sentiment Analysis Preview

In [ ]:
# Sentiment distribution
sentiment_dist = query("""
    SELECT
        CASE
            WHEN sentiment_compound >= 0.05 THEN 'positive'
            WHEN sentiment_compound <= -0.05 THEN 'negative'
            ELSE 'neutral'
        END as sentiment,
        COUNT(*) as count
    FROM fact_social_sentiment
    GROUP BY 1
    ORDER BY count DESC
""")

if not sentiment_dist.empty:
    fig = px.pie(
        sentiment_dist,
        values="count",
        names="sentiment",
        title="Sentiment Distribution",
        color="sentiment",
        color_discrete_map={
            "positive": "#2ecc71",
            "neutral": "#95a5a6",
            "negative": "#e74c3c",
        },
    )
    fig.show()
else:
    print("No sentiment data available yet.")

---

## Next Steps

1. **Explore specific games**: Query `fact_game` joined with `fact_social_sentiment`
2. **Analyze team performance**: Correlate sentiment with game outcomes
3. **Time-series analysis**: Use TimescaleDB functions for time-based aggregations
4. **Build dashboards**: Create interactive visualizations with Plotly

### Key Findings (2025-11-20 Analysis)

**✅ Pipeline is Now Working Correctly**

The root causes have been fixed:
1. ✅ `dim_date` populated (1,461 dates from 2024-2027)
2. ✅ `fact_game` populated (37 games today)
3. ✅ Team aliases populated (173/173 teams)
4. ✅ Transform asset running (576 posts processed)

**❌ Why 0 Posts Are in fact_social_sentiment**

The pipeline requires posts to mention **BOTH teams from the same matchup**:
- Historical posts (March-Nov): Can't match today's games (Nov 20 only)
- Today's posts (90 posts): None mention both teams from a single game
- Example: Posts mention "Colgate" or "Elon" individually, not "Colgate vs Cornell"

**When Matching Will Improve:**

- **Post-game threads**: "Duke defeats UNC 85-78" → mentions both teams
- **High-profile games**: Duke vs UNC, Kansas vs Kentucky get more discussion
- **Tournament games**: March Madness threads explicitly name both teams
- **Live game threads**: Real-time discussion mentions both teams

**Identified Issues to Fix:**

1. **False Positives (High Priority)**:
   - "Colgate" matched from "College" in URLs (36 false matches today)
   - Fix: Increase threshold 70→75, exclude URL text from matching

2. **Overly Strict Matching (Medium Priority)**:
   - Requiring exactly 2 teams excludes valid single-team posts
   - Consider: Allow 1-team posts within ±3 hours of game time

3. **Date Mismatch (Expected Behavior)**:
   - Historical posts can't match today's games
   - Solution: Wait for new posts as season progresses

In [ ]:
# DIAGNOSTIC 2: Date alignment and matching funnel
date_and_matching = query("""
    SELECT
        'Posts (with teams)' as category,
        COUNT(*)::text as count,
        MIN(created_at::date)::text as earliest,
        MAX(created_at::date)::text as latest
    FROM stg_social_posts
    WHERE matched_to_game = true
    UNION ALL
    SELECT
        'Games (available)',
        COUNT(*)::text,
        MIN(game_date::date)::text,
        MAX(game_date::date)::text
    FROM fact_game WHERE sport = 'ncaam'
    UNION ALL
    SELECT
        'Posts (today)',
        COUNT(*)::text,
        CURRENT_DATE::text,
        CURRENT_DATE::text
    FROM stg_social_posts
    WHERE created_at::date = CURRENT_DATE AND matched_to_game = true
    UNION ALL
    SELECT
        'Posts (today, 2 teams)',
        COUNT(*)::text,
        NULL,
        NULL
    FROM stg_social_posts
    WHERE created_at::date = CURRENT_DATE
      AND matched_to_game = true
      AND array_length(matched_teams, 1) = 2
""")

print("=== Date Alignment & Matching Funnel ===")
print("Posts and games must overlap in date for matching to work.")
print("Posts must mention EXACTLY 2 teams to match a specific game.")
date_and_matching

In [ ]:
# DIAGNOSTIC 1: Check pipeline infrastructure
diagnostic_infrastructure = query("""
    SELECT
        'dim_date populated' as check_name,
        CASE WHEN COUNT(*) > 0 THEN '✅ PASS' ELSE '❌ FAIL' END as status,
        COUNT(*)::text || ' dates' as details
    FROM dim_date
    UNION ALL
    SELECT
        'Teams have aliases',
        CASE WHEN COUNT(*) = SUM(CASE WHEN array_length(aliases, 1) > 0 THEN 1 ELSE 0 END)
             THEN '✅ PASS' ELSE '⚠️  PARTIAL' END,
        SUM(CASE WHEN array_length(aliases, 1) > 0 THEN 1 ELSE 0 END)::text || ' / ' || COUNT(*)::text
    FROM dim_team WHERE sport = 'ncaam' AND is_current = true
    UNION ALL
    SELECT
        'Games ingested',
        CASE WHEN COUNT(*) > 0 THEN '✅ PASS' ELSE '❌ FAIL' END,
        COUNT(*)::text || ' games'
    FROM fact_game WHERE sport = 'ncaam'
    UNION ALL
    SELECT
        'Transform asset running',
        CASE WHEN COUNT(*) > 0 THEN '✅ PASS' ELSE '❌ FAIL' END,
        COUNT(*)::text || ' posts'
    FROM stg_social_posts
""")

print("=== Infrastructure Health Checks ===")
diagnostic_infrastructure

## 8. Pipeline Diagnostics & Troubleshooting

### Understanding Why Posts Aren't Matching to Games

The matching pipeline requires **THREE conditions** to be met:
1. ✅ Posts must be matched to teams (via fuzzy text matching)
2. ✅ Games must exist in fact_game for the post date
3. ✅ Post must mention **BOTH teams** from the same matchup

This section helps diagnose which condition is failing.